In [1]:
import numpy as np
import cv2
import torch
import glob as glob
import os
import time
import import_ipynb
from model import create_model
from config import (
    num_classes, device, classes
)

importing Jupyter notebook from model.ipynb
importing Jupyter notebook from config.ipynb


In [2]:
# 為每個標籤建立不同顏色
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [3]:
# Load
model = create_model(num_classes=num_classes)
checkpoint = torch.load('outputs/best_model.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device).eval()

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
dir_test = 'data/PASCAL VOC 2012/test'
test_images = glob.glob(f"{dir_test}/*.jpg")
print(f"Test instances: {len(test_images)}")

Test instances: 4952


In [5]:
# 低於門檻的值將被丟棄
detection_threshold = 0.8

frame_count = 0
total_fps = 0 

In [12]:
for i in range(len(test_images)):
    image_name = test_images[i].split(os.path.sep)[-1].split('.')[0]
    image = cv2.imread(test_images[i])
    orig_image = image.copy()
    # BGR 2 RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    # 將Pixel限制到0~1之間
    image /= 255.0
    # 將color channel放到最前面
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    # convert to tensor
    image = torch.tensor(image, dtype=torch.float).cuda()
    # add batch dimension
    image = torch.unsqueeze(image, 0)
    start_time = time.time()
    with torch.no_grad():
        outputs = model(image.to(device))
    end_time = time.time()
    
    fps = 1 / (end_time - start_time)
    total_fps += fps
    frame_count += 1
    # 使用CPU進行後續操作
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # 檢查是否有偵測到
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        # filter out boxes according to `detection_threshold`
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        print(boxes)
        draw_boxes = boxes.copy()
        # get all the predicited class names
        pred_classes = [classes[i] for i in outputs[0]['labels'].cpu().numpy()]
        
        # draw the bounding boxes and write the class name on top of it
        for j, box in enumerate(draw_boxes):
            class_name = pred_classes[j]
            color = colors[classes.index(class_name)]
            cv2.rectangle(orig_image,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        color, 2)
            cv2.putText(orig_image, class_name, 
                        (int(box[0]), int(box[1]-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 
                        2, lineType=cv2.LINE_AA)
        continue
        cv2.imshow('Prediction', orig_image)
        cv2.waitKey(1)
        cv2.imwrite(f"inference_outputs/images/{image_name}.jpg", orig_image)
    print(f"Image {i+1} done...")
    print('-'*50)
print('TEST PREDICTIONS COMPLETE')
cv2.destroyAllWindows()
# calculate and print the average FPS
avg_fps = total_fps / frame_count
print(f"Average FPS: {avg_fps:.3f}")

[[ 64  14 353 423]]
[[140 201 210 301]]
[]
[[362 325 496 388]
 [232 327 331 375]
 [173 326 245 363]
 [ 16 309  84 363]
 [138 322 183 358]
 [108 325 145 354]
 [ 84 325 115 351]]
[[128 198 246 371]
 [186 187 279 240]
 [277 189 329 247]]
[[198  17 357 254]]
[[118  79 255 272]
 [ 92  79 248 394]]
[[140  56 241 126]
 [140  91 319 283]]
[[299 161 444 251]]
[[ 73 166 318 228]
 [171 178 491 314]]
[[ 75 122 368 349]]
[[ 39  32 354 283]]
[[188  46 252 232]
 [ 75 102 367 279]]
[[313   4 492 373]
 [222   6 347 267]
 [ 54  43 166 194]
 [ 60 149 371 371]]
[[170 101 348 348]]
[[ 16  34 368 495]]
[[ 30  49 289 292]]
[[ 40  87 434 250]]
[[ 98  97 475 321]]
[[188 112 254 237]
 [155 171 275 245]]
[[ 50  68 387 282]]
[[  9 138 297 500]
 [ 42   5 333 427]]
[[  0 134 201 302]
 [267  44 477 255]]
[[180 134 361 227]]
[[  4  25 442 269]]
[[ 17  65 469 287]]
[[219 109 267 173]
 [  0 167 295 330]]
[[161  63 391 334]]
[[147  49 415 373]]
[[326   0 431 375]
 [193  76 300 339]
 [275  53 342 328]
 [ 26 131 378 375]


KeyboardInterrupt: 